In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import json, random, copy
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split

# --- Load data ---
with open("Silver/train_merged_for_selftraining.json", "r") as f:
    merged_labels = json.load(f)

with open("Amazon_products/classes.txt", "r", encoding="utf-8") as f:
    id2label = {int(line.split("\t")[0]): line.split("\t")[1].strip() for line in f}

# suppose you already have your embeddings tensor + pid2idx dict
print("⚙️ Loading embeddings (precomputed)...")
embeddings = torch.load("embeddings.pt")  # shape [N, D]
pid2idx = torch.load("pid2idx.pt")

device = "cuda" if torch.cuda.is_available() else "cpu"
num_classes = len(id2label)
input_dim = embeddings.size(1)

# --- Dataset ---
class MultiLabelDataset(Dataset):
    def __init__(self, pid2labels, pid2idx, embeddings, num_classes):
        self.pids = list(pid2labels.keys())
        self.pid2idx = pid2idx
        self.embeddings = embeddings
        self.num_classes = num_classes
        self.labels = [pid2labels[pid] for pid in self.pids]

    def __len__(self):
        return len(self.pids)

    def __getitem__(self, idx):
        pid = self.pids[idx]
        emb = self.embeddings[self.pid2idx[pid]]
        y = torch.zeros(self.num_classes)
        for lid in self.labels[idx]:
            y[lid] = 1.0
        return {"X": emb, "y": y, "pid": pid}

# Split train/val
pids = list(merged_labels.keys())
train_pids, val_pids = train_test_split(pids, test_size=0.15, random_state=42)
train_labels = {pid: merged_labels[pid] for pid in train_pids}
val_labels = {pid: merged_labels[pid] for pid in val_pids}
print(f"✅ Train: {len(train_labels)} | Val: {len(val_labels)}")

train_ds = MultiLabelDataset(train_labels, pid2idx, embeddings, num_classes)
val_ds = MultiLabelDataset(val_labels, pid2idx, embeddings, num_classes)
train_ld = DataLoader(train_ds, batch_size=64, shuffle=True)
val_ld = DataLoader(val_ds, batch_size=64)

# --- Model ---
class InnerProductClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.proj = nn.Linear(input_dim, 768)
        self.label_emb = nn.Parameter(torch.randn(num_classes, 768))
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.dropout(F.relu(self.proj(x)))
        logits = torch.matmul(x, self.label_emb.T)
        return logits

model = InnerProductClassifier(input_dim, num_classes).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)

# --- Evaluation ---
def evaluate(model, loader):
    model.eval()
    all_preds, all_true = [], []
    with torch.no_grad():
        for batch in loader:
            X, y = batch["X"].to(device), batch["y"].to(device)
            logits = model(X)
            preds = (torch.sigmoid(logits) > 0.5).float()
            all_preds.append(preds.cpu())
            all_true.append(y.cpu())
    all_preds = torch.cat(all_preds)
    all_true = torch.cat(all_true)
    f1 = f1_score(all_true, all_preds, average="micro", zero_division=0)
    return f1

# --- Loss ---
def multilabel_loss(logits, targets):
    return F.binary_cross_entropy_with_logits(logits, targets, reduction="none").sum(dim=1).mean()

# --- Phase 1: base training ---
print("\n=== Phase 1: Base Training ===")
best_f1 = 0
patience, wait = 5, 0
best_model = copy.deepcopy(model.state_dict())

for epoch in range(1, 11):
    model.train()
    total_loss = 0
    for batch in tqdm(train_ld, desc=f"Epoch {epoch}"):
        X, y = batch["X"].to(device), batch["y"].to(device)
        logits = model(X)
        loss = multilabel_loss(logits, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    scheduler.step()
    f1_val = evaluate(model, val_ld)
    print(f"Epoch {epoch:2d} | Loss={total_loss/len(train_ld):.3f} | Val F1={f1_val:.4f}")

    if f1_val > best_f1:
        best_f1 = f1_val
        best_model = copy.deepcopy(model.state_dict())
        wait = 0
        print("  🔥 New best model!")
    else:
        wait += 1
        if wait >= patience:
            print("⏹️ Early stopping.")
            break

model.load_state_dict(best_model)

# --- Save model ---
torch.save(model.state_dict(), "model_selftrain_base.pt")
print("\n💾 Base model saved to model_selftrain_base.pt")

# --- Phase 2 (pseudo-labelling + consistency) ---
print("\n=== Phase 2: Self-training === (multi-label)")

teacher = copy.deepcopy(model)
alpha_ema = 0.999
lambda_cons = 1.5

def ema_update(ema_model, model, alpha):
    for ema_p, p in zip(ema_model.parameters(), model.parameters()):
        ema_p.data.mul_(alpha).add_(p.data, alpha=(1 - alpha))

def consistency_loss(logits_s, logits_t):
    p_s = torch.sigmoid(logits_s)
    p_t = torch.sigmoid(logits_t)
    return F.mse_loss(p_s, p_t)

for epoch in range(1, 21):
    model.train()
    total_loss = 0
    for batch in tqdm(train_ld, desc=f"ST Epoch {epoch}"):
        X, y = batch["X"].to(device), batch["y"].to(device)
        noise = torch.randn_like(X) * 0.05
        logits_s = model(X + noise)
        with torch.no_grad():
            logits_t = teacher(X)
        loss = multilabel_loss(logits_s, y) + lambda_cons * consistency_loss(logits_s, logits_t)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        ema_update(teacher, model, alpha_ema)
        total_loss += loss.item()

    scheduler.step()
    f1_val = evaluate(teacher, val_ld)
    print(f"ST Epoch {epoch:2d} | Loss={total_loss/len(train_ld):.3f} | Val F1={f1_val:.4f}")

print("\n✅ Self-training phase done! Model stabilized.")
torch.save(teacher.state_dict(), "model_selftrain_final.pt")
print("💾 Saved final teacher model -> model_selftrain_final.pt")
